In [14]:
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np
import tensorflow.contrib.rnn as rnn

% matplotlib inline
plt.style.use('ggplot')


In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [32]:
learning_rate = 1e-4
n_epoch = 10
epoch_size = 100
n_iter = n_epoch * epoch_size

batch_size = 100

n_input = 28
n_output = 10
n_steps = 28
n_hidden = 128

In [33]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

W = tf.Variable(tf.truncated_normal([n_hidden, n_output]),dtype=tf.float32)
b = tf.Variable(tf.truncated_normal([n_output]), dtype=tf.float32)

x_unstack = tf.unstack(x, n_steps, 1)


lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

outputs, states = rnn.static_rnn(lstm_cell, x_unstack, dtype=tf.float32)


h = tf.matmul(outputs[-1], W) + b

cost =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=h))
training_steps = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(h,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [34]:
%%time
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()


for iter in range(n_iter+1):
    xs, ys = mnist.train.next_batch(batch_size)
    xs = xs.reshape((batch_size, n_steps, n_input))
    sess.run(training_steps, feed_dict={x:xs, y:ys})
    
    if iter % epoch_size == 0:
        print('Epoch: {} Batch Accuracy: {:4.2f}%'
              .format(int(iter/epoch_size),
               100*accuracy.eval(feed_dict={x:xs, y:ys})))


Epoch: 0 Batch Accuracy: 6.00%
Epoch: 1 Batch Accuracy: 52.00%
Epoch: 2 Batch Accuracy: 64.00%
Epoch: 3 Batch Accuracy: 74.00%
Epoch: 4 Batch Accuracy: 71.00%
Epoch: 5 Batch Accuracy: 79.00%
Epoch: 6 Batch Accuracy: 76.00%
Epoch: 7 Batch Accuracy: 89.00%
Epoch: 8 Batch Accuracy: 84.00%
Epoch: 9 Batch Accuracy: 93.00%
Epoch: 10 Batch Accuracy: 93.00%
CPU times: user 16.9 s, sys: 784 ms, total: 17.7 s
Wall time: 15.1 s


In [42]:
test_x = mnist.test.images
test_y = mnist.test.labels

test_x = test_x.reshape((len(test_y), n_steps, n_input))
a = accuracy.eval(feed_dict={x:test_x, y:test_y})

print('Test Accuracy: {:4.2f}%' .format(a))

Test Accuracy: 0.91%
